In [ ]:
# curl "https://longreads.firebaseio.com/participant_data/.json?print=pretty&auth=TbPc0mqD0pCAEeQKnFzC1trdFH27GfY6z2cKUmum"

In [1]:
import json
import numpy as np
import pandas as pd
import datetime

In [2]:
with open("longreads2-export.json") as json_file:
    json_data = json.load(json_file)

In [17]:
# curr = "45019761"

excludedParticipantIds = ['8aa4a59b']
participantIds = [x for x in json_data.keys() if x not in excludedParticipantIds]
participantIds = pd.Series(participantIds)

# pd.Series(['ee14a4c7', 'bc07be57', '447485a8', '637a20b2', 'c9041366', '1d6d9b1a',
#                             '76eb0f72', 'e76d1c29', '824a6ecd', '21a58161', '5eeea135', '6d48e635',
#                             'cecbbe70', '2e9a547d', 'ef8527a9', '96234b52', '03913d38', 'ba738e46',
#                             '9c82b7b3', '720390ad', 'c6c61167', '6f041d41', 'cad5f235', '02ed1d4a',
#                             'e2d9e6c3', '65c55012', '6c17b273', 'b00cb7ca', '3ddb558f', '54de43e2',
#                             '45019761'])




In [18]:
participantIds

0    8e274deb
1    3e06a4b5
dtype: object

In [5]:
bad_article_url = 'http://munchies.vice.com/articles/new-research-says-vegetarian-diets-could-actually-be-worse-for-the-planet'
def get_article_answers(article):
    article_answers = pd.DataFrame(columns=['condition', 'corr_answer', 'participant_answer', 
                                            'is_correct', 'question_type', 'label'])
    condition = article["experiment condition"]
    
    if condition == "read":
        condNum = 0
    else:
        condNum = 1
    for i, question in enumerate(article["Questions"]):
        pAns = question["answer"]
        cAns = question["correct answer"]
        corr = False
        if pAns == cAns:
            corr = True
        if (article['Url'] == bad_article_url and question['questionID'] == 1):
            if pAns == 3:
                corr = True
            else:
                corr = False
        typeNum = 1
        q_type = question["question type"]
        if q_type == "read":
            typeNum = 0
        lab = 'NoLabel'
        if i % 2 == 1:
            if corr:
                lab = 'in-depth'
            elif (article_answers.loc[i-1]['is_correct']):
                lab = 'skim'
            else:
                lab = 'other'
        article_answers.loc[i] = ([condNum, cAns, pAns, corr, typeNum, lab])
    return article_answers

In [24]:

# self-reported: age, gender, strategy, ..., words per minutes, start/end time
def get_participant_data(idx, partData):
    qual_data_list = []
    part_data_list = []
    for i, article in enumerate(partData['Articles']):
        if i < 2:
            qual_data_list.append(get_article_answers(article))
        else:
            part_data_list.append(get_article_answers(article))
    qual_data_df = pd.concat(qual_data_list)
    total_qual_correct = qual_data_df['is_correct'].value_counts()[True]
    part_data_df = pd.concat(part_data_list)
    total_correct = part_data_df['is_correct'].value_counts()[True]

    total_skim_correct = part_data_df[[x == 1 for x in part_data_df['question_type']]]['is_correct'].value_counts()[True]
    total_in_depth_correct = part_data_df[[x == 0 for x in part_data_df['question_type']]]['is_correct'].value_counts()[True]
    
    
    
    
    try:
        total_skim_labels = part_data_df['label'].value_counts()['skim']
    except KeyError:
        total_skim_labels = 0
    
    
    total_in_depth_labels = part_data_df['label'].value_counts()['in-depth']
    total_other_labels = part_data_df['label'].value_counts()['other']
    reading_sp = partData['Reading Speed (words per sec)']
    part_gen = partData['Begin-Demographic-Questions']['Gender']
    part_age = partData['Begin-Demographic-Questions']['Age']
    part_edu = partData['Begin-Demographic-Questions']['Education']
    part_disability = partData['Begin-Demographic-Questions']['Disability']
    part_diagnosi = partData['Begin-Demographic-Questions']['Diagnosis']
    part_assistive = partData['Demographic-Questions']['Assitive_technology']
    part_start_time = str(datetime.datetime.fromtimestamp(partData['User Info']['exp_start_t'] / 1000.0))
    result_df = pd.DataFrame(columns=['id', 'start_time', 'gender', 'age', 'education', 'word_per_sec', 
                                      'qual_correct', 'total_correct', 'skim_correct', 'in_depth_correct', 
                                      'skim_labels', 'in_depth_labels', 'other_labels', 'strategy'])
    
    
    result_df.loc[idx] = (partData['participant_Id'], part_start_time, part_gen, part_age, part_edu, reading_sp, 
                          total_qual_correct, total_correct, total_skim_correct, total_in_depth_correct, 
                          total_skim_labels, total_in_depth_labels, total_other_labels, part_strategy)
    return result_df

In [22]:
json_data['8aa4a59b']['Begin-Demographic-Questions'].keys()

[u'Gender',
 u'Age',
 u'Disability',
 u'Diagnosis',
 u'Education',
 u'Assitive_technology']

In [7]:
def parse_data():
    all_part = []
    for i, partId in enumerate(participantIds):
        # print i
        # temp = get_participant_data(i, json_data[partId])
        # print temp
        # all_part.append(temp)
        all_part.append(get_participant_data(i, json_data[partId]))
    return pd.concat(all_part)

In [ ]:
# Fixed question issue

In [25]:
ans = parse_data()

KeyError: 'Assitive_technology'

In [ ]:
ans

In [ ]:
ans.to_csv('results.csv')